In [29]:
# import statements
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.tree import export_text
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

In [30]:
# read in preprocessed data
data = pd.read_csv("../data/preprocessed/preprocessed_crime_data.csv")

In [31]:
data.head()

,state,county,community,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop,IncomeInequalityRatio,PoliceToIncomeRatio,EconomicHardshipIndex
0,8,58.826829,46188.336597,1,0.19,0.33,0.02,0.90,0.12,0.17,...,0.060000,0.040000,0.900000,0.500000,0.32,0.140000,0.20,0.344538,0.351351,0.200000
1,53,58.826829,46188.336597,1,0.00,0.16,0.12,0.74,0.45,0.07,...,0.163103,0.076708,0.698589,0.440439,0.00,0.195078,0.67,0.120968,0.701588,0.250000
2,24,58.826829,46188.336597,1,0.00,0.42,0.49,0.56,0.17,0.04,...,0.163103,0.076708,0.698589,0.440439,0.00,0.195078,0.43,0.228346,0.724974,0.363333
3,42,95.000000,6096.000000,1,0.01,0.55,0.02,0.95,0.09,0.05,...,0.163103,0.076708,0.698589,0.440439,0.00,0.195078,0.03,0.358491,0.434984,0.080000
4,44,7.000000,41500.000000,1,0.01,0.39,0.00,0.98,0.06,0.02,...,0.163103,0.076708,0.698589,0.440439,0.00,0.195078,0.03,0.477477,0.517838,0.173333


To predict `ViolentCrimesPerPop` we begin with a Linear regression model for initial exploration, but move to Ridge and Lasso to address overfitting by implementing regularization techniques. 

### Linear Regression 

In [32]:
# Features variable (all but target)
X = data.drop('ViolentCrimesPerPop', axis = 1)

# Target variable
y = data['ViolentCrimesPerPop']

# Split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [33]:
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Only transform the test data using the already fitted scaler
X_test_scaled = scaler.transform(X_test)

In [34]:
# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train_scaled, y_train)

LinearRegression()

In [35]:
# Predict the target variable on the test set
y_pred = model.predict(X_test_scaled)

In [36]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R-squared value
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Squared Error: 1.0118205484151366e+20
R-squared: -3.211050263538348e+21


The results indicate a very large MSE, suggesting that the model is performing very poorly. The negative R squared indicates that the model is misfitting the data severely due to issues such as multicollinearity and overfitting. We now explore Ridge and Lasso regression techniques to address the issue of overfitting and optimize our models. 

### Lasso 

In [37]:
from sklearn.linear_model import Lasso

# Initialize the Lasso model
lasso = Lasso(alpha=0.1) 

# Train the Lasso model on the scaled training data
lasso.fit(X_train_scaled, y_train)

# Predict the target variable on the test set
y_pred = lasso.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Squared Error: 0.031978053060203335
R-squared: -0.014835445546611536


#### Tuning Lasso with Cross-Validation

In [38]:
from sklearn.model_selection import GridSearchCV

# Define a range of alpha values to test
param_grid = {'alpha': np.logspace(-4, 4, 50)}

# Perform GridSearchCV to find the best alpha value
grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Print the best alpha and corresponding performance
print("Best alpha:", grid_search.best_params_['alpha'])
best_lasso = grid_search.best_estimator_

# Predict with the best model
y_pred = best_lasso.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


C:\Users\Samantha\anaconda_work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.552e-02, tolerance: 1.391e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Samantha\anaconda_work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.408e-03, tolerance: 1.381e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Samantha\anaconda_work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

Best alpha: 0.009102981779915217
Mean Squared Error: 0.014151456973300858
R-squared: 0.5508982327474392


With initial alpha=0.1, the MSE is 0.032 suggests that the model's predictions are not accurate and there is still some error. Further, the negative R squared indicates that the model is very poor at explaining the variability in the target variable. We tune the alpha value using grid search for a better outcome. After tuning alpha to = 0.009, our MSE improves to 0.014 while our R squared increases substantially to 0.55, suggesting that the model is better at explaining variance in the target variable with the new alpha value. 

The alpha parameter controls regularization strength and the tuning reveals that a slightly stronger regulartization with a smaller alpha helps reduce overfitting. 

### Ridge 

In [39]:
from sklearn.linear_model import Ridge

# Initialize the Ridge model
ridge = Ridge(alpha=1.0) 

# Train the Ridge model on the scaled training data
ridge.fit(X_train_scaled, y_train)

# Predict the target variable on the test set
y_pred = ridge.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Squared Error: 0.01915710550335068
R-squared: 0.392042108934048


#### Tuning Ridge with Cross-Validation

In [40]:
from sklearn.model_selection import GridSearchCV

# Define a range of alpha values to test
param_grid = {'alpha': np.logspace(-4, 4, 50)}

# Perform GridSearchCV to find the best alpha value
grid_search = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Print the best alpha and corresponding performance
print("Best alpha:", grid_search.best_params_['alpha'])
best_ridge = grid_search.best_estimator_

# Predict with the best model
y_pred = best_ridge.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Best alpha: 494.1713361323828
Mean Squared Error: 0.014031147031348749
R-squared: 0.5547163136454474


When alpha = 0.1, the MSE suggests that the predictions are relatively off from actual values and that the model is not regularizing enough and is overfitting values. The R squared is also low indicating that this model is only moderatly effective at capturing relationships in the data.

After tuning with the best alpha, we find that alpha = 494.171 is the optimal value, meaning regularization is very strong, forcing the model to shrink coefficients more significantly to prevent overfitting by penalizing large coefficients. The MSE is smaller than that of the previous Ridge model as well as the optimized Lasso model above. The R squared value also improved from the first Ridge model. 

The Ridge model using the best alpha yields the best results overall, with a low MSE and high R-squared compared to the linear regression and lasso models. 